In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
from torch import nn
import torch
from torchvision.models.detection.backbone_utils import resnet, BackboneWithFPN, resnet_fpn_backbone
from xv.nn.nets import BoxClassifier
from PIL import Image
import numpy as np
from xv import io

In [2]:
from xv.io import *
conf = Config('config/config-damage-od.yaml')

/home/jupyter/sky-eye/xv/io.py:34: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  _conf = yaml.load(file_path)


In [3]:
train_loader, dev_loader = io.get_damage_loaders(conf)

100%|██████████| 280/280 [00:01<00:00, 170.40it/s]

train/dev instances:  2017 224


In [6]:
encoder_name = 'efficientnet-b0'
encoder = smp.encoders.get_encoder(encoder_name)

In [31]:
FeaturePyramidNetwork

torchvision.ops.feature_pyramid_network.FeaturePyramidNetwork

In [17]:
features = (4,3,2,1)

[32, 24, 40, 112]

In [ ]:
FeaturePyramidNetwork(in_channels_list, out_channels, extra_blocks=None)

In [13]:
out_shapes = smp.encoders.encoders[encoder_name]['out_shapes']

In [51]:
from torch import nn
import segmentation_models_pytorch as smp
from collections import OrderedDict
from torchvision.ops.feature_pyramid_network import FeaturePyramidNetwork

class GeneralFPNBackbone(nn.Module):
    def __init__(self, encoder_name, features = (4,3,2,1), out_channels=256):
        super().__init__()
        self.encoder = smp.encoders.get_encoder(encoder_name)
        self.features = features
        
        out_shapes = smp.encoders.encoders[encoder_name]['out_shapes']
        in_channels = [out_shapes[f] for f in features]
        
        self.out_channels = out_channels
        
    def forward(self, x):
        x = self.encoder(x)
        x = OrderedDict((i, x[i]) for i in self.features)
        return self.fpn(x)

In [54]:
bc = BoxClassifier(GeneralFPNBackbone(encoder_name), 4, featmap_names=(4,3,2,1))

In [55]:
bc.cuda();

In [56]:
with torch.no_grad():
    i,b,c = next(iter(dev_loader))
    out = bc(i.cuda(), [bo.cuda() for bo in b])

In [58]:
out

tensor([[ 0.0470,  0.0478,  0.0043, -0.0298],
        [ 0.0242,  0.0110,  0.0430, -0.0384],
        [ 0.0206,  0.0490,  0.0186, -0.0313],
        ...,
        [-0.0411,  0.0157, -0.0054,  0.0054],
        [ 0.0817,  0.0258,  0.0229, -0.0200],
        [ 0.0403,  0.0015,  0.0563, -0.0272]], device='cuda:0')

In [5]:
import wandb
from xv import io

seg_run_id = "qoijsx0h"
seg_run_path = f"xvr-hlt/sky-eye-full/{seg_run_id}"
conf_file = wandb.restore('config.yaml', run_path=seg_run_path, replace=True).name
state_file = wandb.restore('state_dict.pth', run_path=seg_run_path, replace=True).name


In [4]:
import wandb
dmg_run_id = "zu0uhkyk"
dmg_run_path = f"xvr-hlt/building-damage/{dmg_run_id}"
dmg_conf_file = wandb.restore('config.yaml', run_path=dmg_run_path, replace=True).name
dmg_state_file = wandb.restore('state_dict.pth', run_path=dmg_run_path, replace=True).name

In [5]:
dmg_conf = Config(dmg_conf_file)
dmg_model = io.load_damage_model(dmg_conf, dmg_state_file)

/home/jupyter/sky-eye/xv/io.py:34: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  _conf = yaml.load(file_path)


<All keys matched successfully>


In [6]:
dmg_model = dmg_model.eval().cuda()

In [9]:
from xv.tta import BoxClassifierTTA

dmg_model = BoxClassifierTTA(dmg_model)
dmg_model = dmg_model.cuda().eval()

In [11]:
from xv import run_damage
dev_metrics = run_damage.evaluate(dmg_model, dev_loader, conf.nclasses, loss_fn)


100%|██████████| 56/56 [15:27<00:00, 16.56s/it]


In [12]:
dev_metrics

{'loss': tensor(138.9520, device='cuda:0'),
 'damage:categorical:0:precision': 0.9587116159780452,
 'damage:categorical:0:recall': 0.9731051224975669,
 'damage:categorical:0:f1': 0.9658547479429385,
 'damage:categorical:1:precision': 0.7008871822300613,
 'damage:categorical:1:recall': 0.5891278421648993,
 'damage:categorical:1:f1': 0.640166425134305,
 'damage:categorical:2:precision': 0.7955187922343381,
 'damage:categorical:2:recall': 0.824661151116293,
 'damage:categorical:2:f1': 0.8098278782317169,
 'damage:categorical:3:precision': 0.8502461024329381,
 'damage:categorical:3:recall': 0.8490813247001894,
 'damage:categorical:3:f1': 0.8496633143771124,
 'hmean:damage:categorical:precision': 0.815826867330693,
 'hmean:damage:categorical:recall': 0.7819486526588151,
 'hmean:damage:categorical:f1': 0.7985285939495259,
 'mean:damage:categorical:precision': 0.8263409232188457,
 'mean:damage:categorical:recall': 0.8089938601197372,
 'mean:damage:categorical:f1': 0.8163780914215182}

torch.Size([596, 4])

In [7]:
from torch.nn.modules.loss import CrossEntropyLoss

weights = torch.Tensor(conf.class_weight).float().cuda()
loss_fn = CrossEntropyLoss(weights, reduction=conf.loss_reduce_mode)

100%|██████████| 56/56 [00:18<00:00,  3.09it/s]


In [13]:
dev_metrics

{'loss': tensor(143.7780, device='cuda:0'),
 'damage:categorical:0:precision': 0.9566251158974713,
 'damage:categorical:0:recall': 0.9725116632441376,
 'damage:categorical:0:f1': 0.9645029762757684,
 'damage:categorical:1:precision': 0.6914017186737941,
 'damage:categorical:1:recall': 0.5721420885855113,
 'damage:categorical:1:f1': 0.6261437412790158,
 'damage:categorical:2:precision': 0.7862104802696855,
 'damage:categorical:2:recall': 0.8240910248303941,
 'damage:categorical:2:f1': 0.8047052037966942,
 'damage:categorical:3:precision': 0.8541146268856493,
 'damage:categorical:3:recall': 0.8364122839403997,
 'damage:categorical:3:f1': 0.8451707704212198,
 'hmean:damage:categorical:precision': 0.8106345859347874,
 'hmean:damage:categorical:recall': 0.7714377190617776,
 'hmean:damage:categorical:f1': 0.7905505886059725,
 'mean:damage:categorical:precision': 0.8220879854316501,
 'mean:damage:categorical:recall': 0.8012892651501108,
 'mean:damage:categorical:f1': 0.8101306729431745}

In [14]:
from xv.tta import BoxClassifierTTA

dmg_model_tta = BoxClassifierTTA(dmg_model)

NameError: name 'nn' is not defined

In [11]:
i,b,c = next(iter(dev_loader))

In [13]:
model(i.cuda(),[bb.cuda() for bb in b])

tensor([[-0.0136,  0.0353,  0.0300, -0.0351],
        [-0.0268,  0.0461,  0.0141, -0.0433],
        [-0.0027,  0.0448,  0.0089, -0.0444],
        ...,
        [-0.0289,  0.0241,  0.0323, -0.0026],
        [-0.0144,  0.0195,  0.0299, -0.0213],
        [-0.0103,  0.0369,  0.0282, -0.0394]], device='cuda:0',
       grad_fn=<AddmmBackward>)

In [ ]:
import os
import yaml
import wandb
from glob import glob
from tqdm import tqdm_notebook as tqdm
import numpy as np
import torch
from torch import nn
from xv.util import vis_im_mask
from pprint import pprint
import random
from xv import dataset, io, run_damage
import pandas as pd
import random
from pytorch_toolbelt import losses
from torch.nn.modules.loss import CrossEntropyLoss
from xv.nn.losses import loss_dict, WeightedLoss
from torch.nn.modules.loss import CrossEntropyLoss
from tqdm import tqdm as tqdm
import pdb
import logging
from apex import amp

os.environ['WANDB_MODE'] = 'dryrun'

run_type = 'building-damage'
conf_file = 'config/config-damage-od.yaml'

wandb.init(project=run_type, config=yaml.load(open(conf_file)))
conf = wandb.config
pprint(dict(conf))

train_loader, dev_loader = io.get_damage_loaders(conf)

class MultiScaleResize(nn.Module):
    def __init__(self, scales = (0.5, 0.75, 1.)):
        super().__init__()
        self.scales = scales
    
    @torch.no_grad()
    def forward(self, x, boxes):
        scale = random.choice(self.scales)
        if scale != 1.:
            x = torch.nn.functional.interpolate(x, scale_factor=scale, mode='bilinear', align_corners=False)
            boxes = [b*scale for b in boxes]
        return x, boxes

train_resize = MultiScaleResize(conf.training_scales)

model = io.load_damage_model(conf)
model = model.cuda()

optims = {'adam': torch.optim.Adam}
optim = optims[conf.optim](model.parameters(), lr=conf.lr)

model, optim = amp.initialize(model, optim, opt_level=conf.amp_opt_level)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optim, factor=conf.scheduler_factor, patience=conf.scheduler_patience
)

#loss_fn = WeightedLoss({loss_dict[l]():w for l, w in conf.loss_weights.items()})
#loss_fn = losses.JaccardLoss('multiclass')
#loss_fn = CrossEntropyLoss(weights)

#loss_fn = losses.JointLoss(loss_fn, losses.FocalLoss(), 0.5, 0.5)


epoch, best_score = 0, 0


for epoch in range(epoch, conf.epochs):
    metrics = {'epoch': epoch}
    train_metrics = run_damage.run(model, optim, train_loader, train_resize, loss_fn)
    metrics.update(train_metrics)
    
    
    metrics.update(dev_metrics)
    
    wandb.log(metrics)
    scheduler.step(metrics['loss'])
    score = metrics[conf.metric]
    
    if score > best_score:
        torch.save(model.state_dict(), os.path.join(wandb.run.dir, "state_dict.pth"))
        best_score = score



In [ ]:
im, boxes, clss = train_loader.dataset[0]

In [ ]:
from ttach import functional as tF

In [ ]:
tF.rot90(torch.Tensor(im)).shape

In [ ]:
from xv.io import load_dmg_img

In [ ]:
load_dmg_img??